In [12]:
import pandas as pd

# carregando o dataset
df = pd.read_csv('dataset/steam.csv')
df.shape

(27075, 18)

In [32]:
# list columns and first lines
df.columns
# df.head()

Index(['appid', 'name', 'release_date', 'english', 'developer', 'publisher',
       'platforms', 'required_age', 'categories', 'genres', 'steamspy_tags',
       'achievements', 'positive_ratings', 'negative_ratings',
       'average_playtime', 'median_playtime', 'owners', 'price'],
      dtype='object')

In [13]:
# we will use the collaborative filtering technique
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix

# Seleciona as colunas necessárias
df = df[['genres', 'name', 'positive_ratings', 'steamspy_tags']]

# Transforma o nome do jogo em um ID numérico
le = LabelEncoder()
df['name'] = le.fit_transform(df['name'])

# Transforma as colunas 'genres' e 'steamspy_tags' em IDs numéricos
df['genres'] = df['genres'].str.split(',').apply(lambda x: [s.strip() for s in x])
df['steamspy_tags'] = df['steamspy_tags'].str.split(',').apply(lambda x: [s.strip() for s in x])
genre_le = LabelEncoder()
df['genres'] = df['genres'].apply(lambda x: genre_le.fit_transform(x))
tag_le = LabelEncoder()
df['steamspy_tags'] = df['steamspy_tags'].apply(lambda x: tag_le.fit_transform(x))

# Concatena as colunas 'genres' e 'steamspy_tags' em uma única coluna
df['categories'] = df['genres'].apply(lambda x: ','.join([str(elem) for elem in x])) + ';' + df['steamspy_tags'].apply(lambda x: ','.join([str(elem) for elem in x]))

# Transforma a coluna 'categories' em IDs numéricos
category_le = LabelEncoder()
df['categories'] = category_le.fit_transform(df['categories'])

# Cria a matriz de interações
user_item_matrix = csr_matrix((df['positive_ratings'], (df['categories'], df['name'])))


In [6]:
# Importa as bibliotecas necessárias
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
import numpy as np

# Carrega o dataset
df = pd.read_csv('dataset/steam.csv')

# Seleciona as colunas necessárias
df = df[['name', 'positive_ratings', 'categories', 'genres', 'steamspy_tags']]

# Transforma o nome do jogo em um ID numérico
le = LabelEncoder()
df['name'] = le.fit_transform(df['name'])

# Transforma as colunas 'genres' e 'steamspy_tags' em IDs numéricos
df['genres'] = df['genres'].str.split(',').apply(lambda x: [s.strip() for s in x])
df['steamspy_tags'] = df['steamspy_tags'].str.split(',').apply(lambda x: [s.strip() for s in x])
genre_le = LabelEncoder()
df['genres'] = df['genres'].apply(lambda x: genre_le.fit_transform(x))
tag_le = LabelEncoder()
df['steamspy_tags'] = df['steamspy_tags'].apply(lambda x: tag_le.fit_transform(x))

# Concatena as colunas 'genres' e 'steamspy_tags' em uma única coluna
df['categories'] = df['categories'].str.split(',').apply(lambda x: [s.strip() for s in x])
df['categories'] = df['categories'].apply(lambda x: ','.join([str(elem) for elem in x]))
df['categories'] = LabelEncoder().fit_transform(df['categories'])

# Cria a matriz de interações
user_item_matrix = csr_matrix((df['positive_ratings'], (df['categories'], df['name'])))

# Cria o modelo de recomendação
model = NMF(n_components=30, random_state=42)

# Treina o modelo com a matriz de interações
model.fit(user_item_matrix)

# print(user_item_matrix)
# Função para gerar recomendações
def recommend_games(user_id):
    # Obtém a linha correspondente ao usuário na matriz de interações
    user_row = user_item_matrix.getrow(user_id)

    # Gera as recomendações para o usuário
    recommendations = model.inverse_transform(model.transform(user_row))

    # Converte as recomendações em um array numpy
    recommendations = np.array(recommendations)[0]

    # Obtém os IDs dos jogos recomendados, ordenados por relevância
    recommended_game_ids = np.argsort(-recommendations)[:10]

    # Converte os IDs dos jogos em seus nomes
    recommended_games = le.inverse_transform(recommended_game_ids)

    return recommended_games

# Gera recomendações para o usuário de ID 0
recommended_games = recommend_games(0)

# Imprime os jogos recomendados
print(recommended_games)


["Don't Starve Together" 'Undertale' 'FTL: Faster Than Light' 'Robocraft'
 'War Thunder' 'Arma 3' 'Antichamber' 'Mount & Blade: Warband'
 'Insurgency' 'Subnautica']


In [18]:
# Importa as bibliotecas necessárias, teste com usuário
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
import numpy as np

# Carrega o dataset
df = pd.read_csv('dataset/steam.csv')

# Seleciona as colunas necessárias
df = df[['name', 'positive_ratings', 'categories', 'genres', 'steamspy_tags']]

# Transforma o nome do jogo em um ID numérico
le = LabelEncoder()
df['name'] = le.fit_transform(df['name'])

# Transforma as colunas 'genres' e 'steamspy_tags' em IDs numéricos
df['genres'] = df['genres'].str.split(',').apply(lambda x: [s.strip() for s in x])
df['steamspy_tags'] = df['steamspy_tags'].str.split(',').apply(lambda x: [s.strip() for s in x])
genre_le = LabelEncoder()
df['genres'] = df['genres'].apply(lambda x: genre_le.fit_transform(x))
tag_le = LabelEncoder()
df['steamspy_tags'] = df['steamspy_tags'].apply(lambda x: tag_le.fit_transform(x))

# Concatena as colunas 'genres' e 'steamspy_tags' em uma única coluna
df['categories'] = df['categories'].str.split(',').apply(lambda x: [s.strip() for s in x])
df['categories'] = df['categories'].apply(lambda x: ','.join([str(elem) for elem in x]))
df['categories'] = LabelEncoder().fit_transform(df['categories'])

# Cria a matriz de interações
user_item_matrix = csr_matrix((df['positive_ratings'], (df['categories'], df['name'])))

# Cria o modelo de recomendação
model = NMF(n_components=20, random_state=42)

# Treina o modelo com a matriz de interações
model.fit(user_item_matrix)

# print(user_item_matrix)
# Função para gerar recomendações
def recommend_games(user_id):
    # Obtém a matriz de interações do usuário
    user_interactions = user_item_matrix.getrow(user_id)

    # Cria o modelo de recomendação
    model = NMF(n_components=20, random_state=42)

    # Treina o modelo com as interações do usuário
    model.fit(user_interactions)

    # Gera as recomendações para o usuário
    recommendations = model.inverse_transform(model.transform(user_interactions))

    # Converte as recomendações em um array numpy
    recommendations = np.array(recommendations)[0]

    # Obtém os IDs dos jogos recomendados, ordenados por relevância
    recommended_game_ids = np.argsort(-recommendations)[:10]

    # Converte os IDs dos jogos em seus nomes
    recommended_games = le.inverse_transform(recommended_game_ids)

    return recommended_games

    return recommended_games


# Gera recomendações para o usuário de ID 0
recommended_games = recommend_games(2)

# Imprime os jogos recomendados
for game in recommended_games:
    print(game)
    


S.U.B.
Sword With Sauce
Dead Man's Draw
Star Command Galaxies
Ray of Light
S-COPTER: Trials of Quick Fingers and Logic
Intergalactic Bubbles
Vault Resort
Romance of the Three Kingdoms Ⅲ / 三國志Ⅲ
Once10


In [60]:
# Importa as bibliotecas necessárias, teste com usuario Matheus
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF
import numpy as np
import random

# Define a função para gerar recomendações
def recommend_games(user_interactions):
    # Cria o modelo de recomendação
    model = NMF(n_components=20, random_state=random.randint(0, 9999))

    # Treina o modelo com as interações do usuário
    model.fit(user_interactions)

    # Gera as recomendações para o usuário
    recommendations = model.inverse_transform(np.clip(model.transform(user_interactions), 0, np.inf))

    # Converte as recomendações em um array numpy
    recommendations = np.array(recommendations)[0]

    # Obtém os IDs dos jogos recomendados, ordenados por relevância
    recommended_game_ids = np.argsort(-recommendations)[:10]

    # Converte os IDs dos jogos em seus nomes
    recommended_games = le.inverse_transform(recommended_game_ids)

    return recommended_games

# Carrega o dataset
df = pd.read_csv('dataset/steam.csv')

# Seleciona as colunas necessárias
df = df[['name', 'positive_ratings', 'categories', 'genres', 'steamspy_tags']]

# Transforma o nome do jogo em um ID numérico
le = LabelEncoder()
df['name'] = le.fit_transform(df['name'])

genre_le = LabelEncoder()
tag_le = LabelEncoder()
# Transforma as colunas 'genres' e 'steamspy_tags' em IDs numéricos
df['genres'] = df['genres'].str.split(',').apply(lambda x: [s.strip() for s in x])
df['genres'] = df['genres'].apply(lambda x: genre_le.fit_transform(x))

df['steamspy_tags'] = df['steamspy_tags'].str.split(',').apply(lambda x: [s.strip() for s in x])
df['steamspy_tags'] = df['steamspy_tags'].apply(lambda x: tag_le.fit_transform(x))

# Concatena as colunas 'genres' e 'steamspy_tags' em uma única coluna
df['categories'] = df['categories'].str.split(',').apply(lambda x: [s.strip() for s in x])
df['categories'] = df['categories'].apply(lambda x: ','.join([str(elem) for elem in x]))
df['categories'] = LabelEncoder().fit_transform(df['categories'])

# Cria o perfil do usuário Matheus
matheus_profile = pd.DataFrame({
                                "name": ["Counter-Strike", "Call of Duty®"],                              
                                "categories": ["Multi-player;Online Multi-Local Multi-Player;Valve Anti-Cheat enabled", "Single-player;Multi-player"],
                                "genres": ["Action", "Action"],
                                "steamspy_tags": ["Action;FPS;Multiplayer", "FPS;World War II;Action"],                      
                                "positive_ratings": [124534, 1555],
                                })

# Transforma o nome do jogo em um ID numérico
matheus_profile['name'] = le.transform(matheus_profile['name'])

# Transforma as colunas 'genres' e 'steamspy_tags' em IDs numéricos
# Transforma as colunas 'genres' e 'steamspy_tags' em IDs numéricos
matheus_profile['genres'] = matheus_profile['genres'].str.split(',').apply(lambda x: [s.strip() for s in x])
matheus_profile['genres'] = matheus_profile['genres'].apply(lambda x: genre_le.fit_transform(x))

matheus_profile['steamspy_tags'] = matheus_profile['steamspy_tags'].str.split(',').apply(lambda x: [s.strip() for s in x])
matheus_profile['steamspy_tags'] = matheus_profile['steamspy_tags'].apply(lambda x: tag_le.fit_transform(x))


# Concatena as colunas 'genres' e 'steamspy_tags' em uma única coluna
matheus_profile['categories'] = matheus_profile['categories'].str.split(',').apply(lambda x: [s.strip() for s in x])
matheus_profile['categories'] = matheus_profile['categories'].apply(lambda x: ','.join([str(elem) for elem in x]))
matheus_profile['categories'] = LabelEncoder().fit_transform(matheus_profile['categories'])

# Concatena as informações do perfil do usuário com o dataframe original
df = pd.concat([df, matheus_profile], ignore_index=True)

# Cria a matriz de interações do usuário Matheus
user_item_matrix = csr_matrix((df['positive_ratings'], (df['categories'], df['name'])))
matheus_interactions = user_item_matrix[-2, :]

# Gera recomendações para o usuário Matheus
recommended_games = recommend_games(matheus_interactions)

# Imprime os jogos recomendados
print("Jogos recomendados para o usuário Matheus:")
for game in recommended_games:
    print(game) 


Jogos recomendados para o usuário Matheus:
Virtual Desktop
Nimbatus - The Space Drone Constructor
Moorhuhn / Crazy Chicken Tales
Joyo Kanji Quiz
THE QUIET MAN™
Taimumari
THE KING OF FIGHTERS XIV STEAM EDITION
Landless
Boxplosion
SWAT 3: Tactical Game of the Year Edition


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from sklearn.decomposition import NMF

# Carregando o dataset de exemplo
df = pd.read_csv('dataset/steam.csv')

# Criando o LabelEncoder para a coluna 'name'
le = LabelEncoder()
df['name'] = le.fit_transform(df['name'])

# Criando o LabelEncoder para as colunas 'categories', 'steamspy_tags' e 'genres'
le_categories = LabelEncoder()
le_tags = LabelEncoder()
le_genres = LabelEncoder()

# Transformando os valores das colunas em códigos numéricos
df['categories'] = df['categories'].str.split(
    ';').apply(lambda x: [s.strip() for s in x])
df['categories'] = df['categories'].apply(
    lambda x: ','.join([str(elem) for elem in x]))
df['categories'] = le_categories.fit_transform(df['categories'])

df['steamspy_tags'] = df['steamspy_tags'].str.split(
    ';').apply(lambda x: [s.strip() for s in x])
df['steamspy_tags'] = df['steamspy_tags'].apply(
    lambda x: ','.join([str(elem) for elem in x]))
df['steamspy_tags'] = le_tags.fit_transform(df['steamspy_tags'])

df['genres'] = df['genres'].str.split(
    ';').apply(lambda x: [s.strip() for s in x])
df['genres'] = df['genres'].apply(
    lambda x: ','.join([str(elem) for elem in x]))
df['genres'] = le_genres.fit_transform(df['genres'])

# Criando a matriz de usuário-item
user_item_matrix = csr_matrix((df['positive_ratings'], (df['categories'], df['name'])), shape=(
    len(le_categories.classes_), len(le.classes_)))

# Treinando o modelo NMF
model = NMF(n_components=10, random_state=42)
model.fit(user_item_matrix)

# Função para transformar o perfil do usuário


def transform_user_profile(user_profile):
    # Transformando os valores das colunas em códigos numéricos
    user_profile['name'] = le.transform(user_profile['name'])

    user_profile['categories'] = user_profile['categories'].str.split(
        ';').apply(lambda x: [s.strip() for s in x])
    user_profile['categories'] = user_profile['categories'].apply(
        lambda x: ','.join([str(elem) for elem in x]))
    user_profile['categories'] = le_categories.transform(
        user_profile['categories'])

    user_profile['steamspy_tags'] = user_profile['steamspy_tags'].str.split(
        ';').apply(lambda x: [s.strip() for s in x])
    user_profile['steamspy_tags'] = user_profile['steamspy_tags'].apply(
        lambda x: ','.join([str(elem) for elem in x]))
    user_profile['steamspy_tags'] = le_tags.transform(
        user_profile['steamspy_tags'])

    user_profile['genres'] = user_profile['genres'].str.split(
        ';').apply(lambda x: [s.strip() for s in x])
    user_profile['genres'] = user_profile['genres'].apply(
        lambda x: ','.join([str(elem) for elem in x]))
    user_profile['genres'] = le_genres.transform(user_profile['genres'])

    return user_profile

# Função para gerar recomendações para o perfil do usuário


def recommend_games_from_profile(user_profile):
    user_profile_transformed = transform_user_profile(user_profile)

    user_item_matrix_profile = csr_matrix((user_profile_transformed['positive_ratings'],
                                        (user_profile_transformed['categories'], user_profile_transformed['name'])),
                                        shape=(len(le_categories.classes_), len(le.classes_)))

    user_item_matrix_profile_tags = csr_matrix((user_profile_transformed['positive_ratings'],
                                                    (user_profile_transformed['steamspy_tags'], user_profile_transformed['name'])),
                                                shape=(len(le_tags.classes_), len(le.classes_)))

    user_item_matrix_profile_genres = csr_matrix((user_profile_transformed['positive_ratings'],
                                                    (user_profile_transformed['genres'], user_profile_transformed['name'])),
                                                    shape=(len(le_genres.classes_), len(le.classes_)))


    # Ajustando a forma das matrizes
    user_item_matrix_profile_tags_adjusted = csr_matrix((user_item_matrix_profile_tags.data, user_item_matrix_profile_tags.indices,
                                                        user_item_matrix_profile_tags.indptr),
                                                        shape=user_item_matrix_profile.shape)
    user_item_matrix_profile_tags_adjusted.resize(user_item_matrix_profile.shape)

    user_item_matrix_profile_genres_adjusted = csr_matrix((user_item_matrix_profile_genres.data, user_item_matrix_profile_genres.indices,
                                                        user_item_matrix_profile_genres.indptr),
                                                        shape=user_item_matrix_profile.shape)

    # Somando as matrizes
    user_item_matrix_profile = user_item_matrix_profile + user_item_matrix_profile_tags_adjusted + user_item_matrix_profile_genres_adjusted


    # Gerando as recomendações
    recommendations = model.inverse_transform(
        model.transform(user_item_matrix_profile))

    # Calculando a média das recomendações para cada jogo
    recommendations = np.array(recommendations).mean(axis=0)

    # Ordenando os jogos recomendados por ordem decrescente de probabilidade de serem recomendados
    recommended_game_ids = np.argsort(-recommendations)[:10]

    # Transformando os IDs numéricos dos jogos recomendados em seus nomes originais
    recommended_games = le.inverse_transform(recommended_game_ids)

    return recommended_games


# Perfil do usuário Matheus
matheus_profile = pd.DataFrame({"appid": [10, 2620],
                                "name": ["Counter-Strike", "Call of Duty®"],
                                "release_date": ["2000-11-01", "2006-10-13"],
                                "english": [1, 1],
                                "developer": ["Valve", "Infinity Ward"],
                                "publisher": ["Valve", "Activision"],
                                "platforms": ["windows;mac;linux", "windows"],
                                "required_age": [0, 0],
                                "categories": ["Multi-player;Online Multi-Player;Local Multi-Player;Valve Anti-Cheat enabled", "Single-player;Multi-player"],
                                "genres": ["Action", "Action"],
                                "steamspy_tags": ["Action;FPS;Multiplayer", "FPS;World War II;Action"],
                                "achievements": [0, 0],
                                "positive_ratings": [124534, 1555],
                                "negative_ratings": [3339, 117],
                                "average_playtime": [17612, 545],
                                "median_playtime": [317, 849],
                                "owners": ["10000000-20000000", "200000-500000"],
                                "price": [7.19, 14.99]
                                })

# Gera recomendações para o usuário Matheus
recommended_games = recommend_games_from_profile(matheus_profile)

# Imprime os jogos recomendados
print(recommended_games)
